In [1]:
import pandas as pd
import os
import sys
sys.path.insert(0, os.path.abspath('../lib'))
from c19us_nyt import df_us as df_nyt
from c19us_jhu import df_us as df_jhu
import numpy as np

In [2]:
df_nyt = df_nyt.reset_index()[['date','fips','day','county','state','cases','deaths']]
print(df_nyt.shape)
df_nyt.head()

(91470, 7)


,date,fips,day,county,state,cases,deaths
0,2020-01-21,53061,0,Snohomish,Washington,1,0
1,2020-01-22,53061,1,Snohomish,Washington,1,0
2,2020-01-23,53061,2,Snohomish,Washington,1,0
3,2020-01-24,17031,3,Cook,Illinois,1,0
4,2020-01-24,53061,3,Snohomish,Washington,1,0


In [3]:
df_nyt = df_nyt.groupby(['date','fips','day','county','state']).sum().reset_index()
print(df_nyt.shape)
df_nyt.head()

(91470, 7)


,date,fips,day,county,state,cases,deaths
0,2020-01-21,53061,0,Snohomish,Washington,1,0
1,2020-01-22,53061,1,Snohomish,Washington,1,0
2,2020-01-23,53061,2,Snohomish,Washington,1,0
3,2020-01-24,17031,3,Cook,Illinois,1,0
4,2020-01-24,53061,3,Snohomish,Washington,1,0


In [4]:
df_jhu = df_jhu.reset_index()[['date','fips','day','county','state','cases','deaths','recovered','active']]
print(df_jhu.shape)
df_jhu.head()

(98390, 9)


,date,fips,day,county,state,cases,deaths,recovered,active
0,2020-03-22,36061,0,New York City,New York,9654,63,0,0
1,2020-03-22,36059,0,Nassau,New York,1900,4,0,0
2,2020-03-22,36119,0,Westchester,New York,1873,0,0,0
3,2020-03-22,36103,0,Suffolk,New York,1034,9,0,0
4,2020-03-22,36087,0,Rockland,New York,455,1,0,0


In [5]:
df_jhu = df_jhu.groupby(['date','fips','day','county','state']).sum().reset_index()
print(df_jhu.shape)
df_jhu.head()

(98375, 9)


,date,fips,day,county,state,cases,deaths,recovered,active
0,2020-03-22,1001,0,Autauga,Alabama,0,0,0,0
1,2020-03-22,1003,0,Baldwin,Alabama,2,0,0,0
2,2020-03-22,1005,0,Barbour,Alabama,0,0,0,0
3,2020-03-22,1007,0,Bibb,Alabama,0,0,0,0
4,2020-03-22,1009,0,Blount,Alabama,0,0,0,0


In [6]:
states = pd.read_csv('../../covid-19/data/state_codes.csv')
states.head()

,Code,State
0,AL,Alabama
1,AK,Alaska
2,AZ,Arizona
3,AR,Arkansas
4,CA,California


In [7]:
df = pd.merge(df_nyt, df_jhu, on=['date','fips','day','county','state'], how='outer')\
    .sort_values('date')
print(df.shape)
df.head()

(189845, 11)


,date,fips,day,county,state,cases_x,deaths_x,cases_y,deaths_y,recovered,active
0,2020-01-21,53061,0,Snohomish,Washington,1.0,0.0,NaN,NaN,NaN,NaN
1,2020-01-22,53061,1,Snohomish,Washington,1.0,0.0,NaN,NaN,NaN,NaN
2,2020-01-23,53061,2,Snohomish,Washington,1.0,0.0,NaN,NaN,NaN,NaN
3,2020-01-24,17031,3,Cook,Illinois,1.0,0.0,NaN,NaN,NaN,NaN
4,2020-01-24,53061,3,Snohomish,Washington,1.0,0.0,NaN,NaN,NaN,NaN


In [8]:
# df

In [9]:
df.loc[df.county=='New York City','cases_x'] = df.loc[df.county=='New York City','cases_y']

In [10]:
df = df[(df.cases_x.isna()==False)]

In [11]:
df = df[df.cases_x.isna()==False]

In [12]:
df['cases'] = df['cases_y'].fillna(df['cases_x'])
df['deaths'] = df['deaths_y'].fillna(df['deaths_x'])
df = df.drop(['cases_x','cases_y','deaths_x','deaths_y'], axis=1)
df['recovered'] = df['recovered'].map(lambda x: np.nan if np.isnan(x) else int(x))
df['active'] = df['active'].map(lambda x: np.nan if np.isnan(x) else int(x))
df['cases'] = df['cases'].map(lambda x: np.nan if np.isnan(x) else int(x))
df['deaths'] = df['deaths'].map(lambda x: np.nan if np.isnan(x) else int(x))
print(df.shape)
df.head()

(91506, 9)


,date,fips,day,county,state,recovered,active,cases,deaths
0,2020-01-21,53061,0,Snohomish,Washington,NaN,NaN,1,0
1,2020-01-22,53061,1,Snohomish,Washington,NaN,NaN,1,0
2,2020-01-23,53061,2,Snohomish,Washington,NaN,NaN,1,0
3,2020-01-24,17031,3,Cook,Illinois,NaN,NaN,1,0
4,2020-01-24,53061,3,Snohomish,Washington,NaN,NaN,1,0


In [13]:
df = df.reset_index()[['date','fips','day','county','state','recovered','active','cases','deaths']]
df.columns = ['Date','FIPS','Day','County','State','Recovered','Active','Cases','Deaths']
print(df.shape)
df.head()

(91506, 9)


,Date,FIPS,Day,County,State,Recovered,Active,Cases,Deaths
0,2020-01-21,53061,0,Snohomish,Washington,NaN,NaN,1,0
1,2020-01-22,53061,1,Snohomish,Washington,NaN,NaN,1,0
2,2020-01-23,53061,2,Snohomish,Washington,NaN,NaN,1,0
3,2020-01-24,17031,3,Cook,Illinois,NaN,NaN,1,0
4,2020-01-24,53061,3,Snohomish,Washington,NaN,NaN,1,0


In [42]:
df_ = pd.merge(df,states,on='State',how='left')

In [43]:
# indexNames = df_[df_['County'].isin(['Bronx','Queens','Manhattan','Staten Island','Brooklyn'])].index
# df_.drop(indexNames , inplace=True)

In [94]:
all_counties = pd.read_csv('../../covid-19/data/county_fips.csv')
all_counties['County'] = all_counties.County.map(lambda x: x.replace('County','').strip())
all_counties.head()

,FIPS,County,Code
0,0,UNITED STATES,NaN
1,1000,ALABAMA,NaN
2,1001,Autauga,AL
3,1003,Baldwin,AL
4,1005,Barbour,AL


In [45]:
all_counties.loc[all_counties['County']=='Queens',:]

,FIPS,County,Code
1902,36081,Queens,NY


In [46]:
all_counties.loc[all_counties['County']=='Bronx',:]

,FIPS,County,Code
1864,36005,Bronx,NY


In [47]:
all_counties.loc[all_counties['County']=='Kings',:]

,FIPS,County,Code
207,6031,Kings,CA
1885,36047,Kings,NY


In [48]:
all_counties.loc[all_counties['County']=='Richmond',:]

,FIPS,County,Code
519,13245,Richmond,GA
1904,36085,Richmond,NY
2001,37153,Richmond,NC
2943,51159,Richmond,VA


In [49]:
all_counties[all_counties['FIPS']==36061]

,FIPS,County,Code
1892,36061,New York,NY


In [95]:
all_counties.loc[all_counties['FIPS']==36061,'County'] = 'Manhattan'

In [96]:
NY_Burrows = ['Manhattan','Queens','Kings','Bronx','Richmond']
NY_Burrow_fips = [36061,36081,36047,36005,36085]

for d in sorted(set(df_['Date'])):
    print('Testing... ' + f'../../covid-19/data/county_data/date_{str(d).split(" ")[0]}.csv')
    dff = df_[df_['Date'] == d]
    if 'New York City' in dff['County'].tolist():
        for c, f in zip(NY_Burrows, NY_Burrow_fips):
            new_row = dff.copy()[dff['County']=='New York City'].reset_index(drop=True)
            new_row.loc[0,['FIPS',
                           'County',
                           'Recovered',
                           'Active',
                           'Cases',
                           'Deaths']] = [f,
                                          c, 
                                          new_row.loc[0,'Recovered']/5,
                                          new_row.loc[0,'Active']/5,
                                          new_row.loc[0,'Cases']/5,
                                          new_row.loc[0,'Deaths']/5
                                         ]
#             print(new_row)
            dff = dff.append(new_row, ignore_index=True)
        dff = dff.loc[np.where(dff['County']!='New York City')]
#         print(dff[dff['Code']=='NY'].sort_values(by='County'))
        
    added_counties = pd.merge(dff, all_counties, how='right', on='FIPS')
    print(added_counties.loc[added_counties['County_x'].isin(NY_Burrows+['New York City'])])
    added_counties = added_counties.drop(['County_x','Code_x'], axis=1)
    added_counties.columns = ['Date','FIPS','Day','State','Recovered','Active','Cases','Deaths','County','Code']
    added_counties['Cases'] = added_counties['Cases'].map(lambda x: 0 if np.isnan(x) else int(x))
    added_counties['Deaths'] = added_counties['Deaths'].map(lambda x: 0 if np.isnan(x) else int(x))
    added_counties['FIPS'] = added_counties['FIPS'].map(lambda x: '0' + str(x) if (int(x) <= 9999) else str(x))

    added_counties.to_csv(f'../../covid-19/data/county_data/date_{str(d).split(" ")[0]}.csv', index=False)

Testing... ../../covid-19/data/county_data/date_2020-01-21.csv
Empty DataFrame
Columns: [Date, FIPS, Day, County_x, State, Recovered, Active, Cases, Deaths, Code_x, County_y, Code_y]
Index: []
Testing... ../../covid-19/data/county_data/date_2020-01-22.csv
Empty DataFrame
Columns: [Date, FIPS, Day, County_x, State, Recovered, Active, Cases, Deaths, Code_x, County_y, Code_y]
Index: []
Testing... ../../covid-19/data/county_data/date_2020-01-23.csv
Empty DataFrame
Columns: [Date, FIPS, Day, County_x, State, Recovered, Active, Cases, Deaths, Code_x, County_y, Code_y]
Index: []
Testing... ../../covid-19/data/county_data/date_2020-01-24.csv
Empty DataFrame
Columns: [Date, FIPS, Day, County_x, State, Recovered, Active, Cases, Deaths, Code_x, County_y, Code_y]
Index: []
Testing... ../../covid-19/data/county_data/date_2020-01-25.csv
Empty DataFrame
Columns: [Date, FIPS, Day, County_x, State, Recovered, Active, Cases, Deaths, Code_x, County_y, Code_y]
Index: []
Testing... ../../covid-19/data/coun

Testing... ../../covid-19/data/county_data/date_2020-03-05.csv
Empty DataFrame
Columns: [Date, FIPS, Day, County_x, State, Recovered, Active, Cases, Deaths, Code_x, County_y, Code_y]
Index: []
Testing... ../../covid-19/data/county_data/date_2020-03-06.csv
Empty DataFrame
Columns: [Date, FIPS, Day, County_x, State, Recovered, Active, Cases, Deaths, Code_x, County_y, Code_y]
Index: []
Testing... ../../covid-19/data/county_data/date_2020-03-07.csv
Empty DataFrame
Columns: [Date, FIPS, Day, County_x, State, Recovered, Active, Cases, Deaths, Code_x, County_y, Code_y]
Index: []
Testing... ../../covid-19/data/county_data/date_2020-03-08.csv
Empty DataFrame
Columns: [Date, FIPS, Day, County_x, State, Recovered, Active, Cases, Deaths, Code_x, County_y, Code_y]
Index: []
Testing... ../../covid-19/data/county_data/date_2020-03-09.csv
Empty DataFrame
Columns: [Date, FIPS, Day, County_x, State, Recovered, Active, Cases, Deaths, Code_x, County_y, Code_y]
Index: []
Testing... ../../covid-19/data/coun

Testing... ../../covid-19/data/county_data/date_2020-03-28.csv
           Date   FIPS   Day   County_x           State  Recovered  Active  \
172  2020-03-28  37153  67.0   Richmond  North Carolina        NaN     NaN   
1400 2020-03-28  13245  67.0   Richmond         Georgia        NaN     NaN   
1604 2020-03-28   6031  67.0      Kings      California        NaN     NaN   
1868 2020-03-28  36061   6.0  Manhattan        New York        0.0     0.0   
1869 2020-03-28  36081   6.0     Queens        New York        0.0     0.0   
1870 2020-03-28  36047   6.0      Kings        New York        0.0     0.0   
1871 2020-03-28  36005   6.0      Bronx        New York        0.0     0.0   
1872 2020-03-28  36085   6.0   Richmond        New York        0.0     0.0   

       Cases  Deaths Code_x   County_y Code_y  
172      1.0     0.0     NC   Richmond     NC  
1400    12.0     0.0     GA   Richmond     GA  
1604     1.0     0.0     CA      Kings     CA  
1868  5955.2   103.4     NY  Manhattan    

           Date   FIPS   Day   County_x       State  Recovered  Active  \
87   2020-04-04   6031  74.0      Kings  California        NaN     NaN   
1722 2020-04-04  13245  74.0   Richmond     Georgia        NaN     NaN   
2266 2020-04-04  51159  74.0   Richmond    Virginia        NaN     NaN   
2373 2020-04-04  36061  13.0  Manhattan    New York        0.0     0.0   
2374 2020-04-04  36081  13.0     Queens    New York        0.0     0.0   
2375 2020-04-04  36047  13.0      Kings    New York        0.0     0.0   
2376 2020-04-04  36005  13.0      Bronx    New York        0.0     0.0   
2377 2020-04-04  36085  13.0   Richmond    New York        0.0     0.0   

        Cases  Deaths Code_x   County_y Code_y  
87        5.0     0.0     CA      Kings     CA  
1722     51.0     1.0     GA   Richmond     GA  
2266      1.0     0.0     VA   Richmond     VA  
2373  12661.2   381.0     NY  Manhattan     NY  
2374  12661.2   381.0     NY     Queens     NY  
2375  12661.2   381.0     NY      Kings

Testing... ../../covid-19/data/county_data/date_2020-04-12.csv
           Date   FIPS   Day   County_x           State  Recovered   Active  \
959  2020-04-12   6031  82.0      Kings      California        NaN      NaN   
1150 2020-04-12  13245  82.0   Richmond         Georgia        NaN      NaN   
1897 2020-04-12  51159  82.0   Richmond        Virginia        NaN      NaN   
2044 2020-04-12  37153  82.0   Richmond  North Carolina        NaN      NaN   
2649 2020-04-12  36061  21.0  Manhattan        New York        0.0  19262.0   
2650 2020-04-12  36081  21.0     Queens        New York        0.0  19262.0   
2651 2020-04-12  36047  21.0      Kings        New York        0.0  19262.0   
2652 2020-04-12  36005  21.0      Bronx        New York        0.0  19262.0   
2653 2020-04-12  36085  21.0   Richmond        New York        0.0  19262.0   

        Cases  Deaths Code_x   County_y Code_y  
959      10.0     1.0     CA      Kings     CA  
1150    144.0     4.0     GA   Richmond     GA  

           Date   FIPS   Day   County_x           State  Recovered   Active  \
125  2020-04-19   6031  89.0      Kings      California        NaN      NaN   
270  2020-04-19  13245  89.0   Richmond         Georgia        NaN      NaN   
1283 2020-04-19  51159  89.0   Richmond        Virginia        NaN      NaN   
2397 2020-04-19  37153  89.0   Richmond  North Carolina        NaN      NaN   
2740 2020-04-19  36061  28.0  Manhattan        New York        0.0  24849.8   
2741 2020-04-19  36081  28.0     Queens        New York        0.0  24849.8   
2742 2020-04-19  36047  28.0      Kings        New York        0.0  24849.8   
2743 2020-04-19  36005  28.0      Bronx        New York        0.0  24849.8   
2744 2020-04-19  36085  28.0   Richmond        New York        0.0  24849.8   

        Cases  Deaths Code_x   County_y Code_y  
125      30.0     1.0     CA      Kings     CA  
270     264.0    10.0     GA   Richmond     GA  
1283      9.0     0.0     VA   Richmond     VA  
2397     19.0

In [20]:
df2 = df[['Date','Day','State','Recovered','Active','Cases','Deaths']]\
    .groupby(['Date','State'])\
    .sum()\
    .reset_index() #.drop('Day', axis=1)

In [21]:
df3 = pd.merge(df2, states, how='left', on='State')
df3['Recovered'] = df3['Recovered'].map(lambda x: int(x))
df3['Active'] = df3['Active'].map(lambda x: int(x))

In [22]:
print(df3.shape)
df3.head()

(2850, 8)


,Date,State,Day,Recovered,Active,Cases,Deaths,Code
0,2020-01-21,Washington,0,0,0,1,0,WA
1,2020-01-22,Washington,1,0,0,1,0,WA
2,2020-01-23,Washington,2,0,0,1,0,WA
3,2020-01-24,Illinois,3,0,0,1,0,IL
4,2020-01-24,Washington,3,0,0,1,0,WA


In [23]:
df3.to_csv('../../covid-19/data/daily_cases_USA_states.csv', index=False)

In [24]:
print(pd.read_csv('../../covid-19/data/daily_cases_USA_states.csv').shape)
pd.read_csv('../../covid-19/data/daily_cases_USA_states.csv').head()

(2850, 8)


,Date,State,Day,Recovered,Active,Cases,Deaths,Code
0,2020-01-21,Washington,0,0,0,1,0,WA
1,2020-01-22,Washington,1,0,0,1,0,WA
2,2020-01-23,Washington,2,0,0,1,0,WA
3,2020-01-24,Illinois,3,0,0,1,0,IL
4,2020-01-24,Washington,3,0,0,1,0,WA


In [25]:
df4 = df_[['Date', 'County','Recovered','Active','Cases','Deaths','Code']]\
    .groupby(['Date','County','Code'])\
    .sum()\
    .reset_index()
df4['Recovered'] = df4['Recovered'].map(lambda x: int(x))
df4['Active'] = df4['Active'].map(lambda x: int(x))

In [26]:
print(df4.shape)
df4.head()

(91455, 7)


,Date,County,Code,Recovered,Active,Cases,Deaths
0,2020-01-21,Snohomish,WA,0,0,1,0
1,2020-01-22,Snohomish,WA,0,0,1,0
2,2020-01-23,Snohomish,WA,0,0,1,0
3,2020-01-24,Cook,IL,0,0,1,0
4,2020-01-24,Snohomish,WA,0,0,1,0


In [27]:
df4.to_csv('../../covid-19/data/daily_cases_USA_counties.csv', index=False)

In [28]:
print(pd.read_csv('../../covid-19/data/daily_cases_USA_counties.csv').shape)
pd.read_csv('../../covid-19/data/daily_cases_USA_counties.csv').head()

(91455, 7)


,Date,County,Code,Recovered,Active,Cases,Deaths
0,2020-01-21,Snohomish,WA,0,0,1,0
1,2020-01-22,Snohomish,WA,0,0,1,0
2,2020-01-23,Snohomish,WA,0,0,1,0
3,2020-01-24,Cook,IL,0,0,1,0
4,2020-01-24,Snohomish,WA,0,0,1,0


In [29]:
df4.Date.min(), df4.Date.max()

(Timestamp('2020-01-21 00:00:00'), Timestamp('2020-04-26 00:00:00'))

In [30]:
df4[df4['Date']=='2020-04-22'].Cases.sum()

827958